In [ ]:
# Analysis: aggregate majority signal per (date, symbol) from `stock.csv`, then backtest using `price.csv` (open->close intraday)
# Steps:
# 1) Load cleaned `stock.csv` and `price.csv` (provided).
# 2) For each date+symbol, count Buy/Hold/Sell and take majority; ties -> Hold.
# 3) For Buy signals: buy at Open, sell at Close. For Sell signals: short at Open, buy to cover at Close.
# 4) Compute return% and PnL assuming CAPITAL_PER_TRADE.
# 5) Save `analysis_results.csv` and print summary + per-symbol breakdown.

import pandas as pd
import os
from datetime import datetime
import math

CAPITAL_PER_TRADE = 1000.0

# 1) load files
stock_path = os.path.join(os.getcwd(), 'stock.csv')
price_path = os.path.join(os.getcwd(), 'price.csv')
if not os.path.exists(stock_path):
    raise FileNotFoundError(f'stock.csv not found at {stock_path}')
if not os.path.exists(price_path):
    raise FileNotFoundError(f'price.csv not found at {price_path}')

stock = pd.read_csv(stock_path, parse_dates=['date', 'published'], keep_default_na=False)
price = pd.read_csv(price_path, parse_dates=['date'])

# normalize recommendation
stock['recommendation'] = stock['recommendation'].astype(str).str.title().str.strip()
stock = stock[stock['recommendation'].isin(['Buy','Sell','Hold'])].copy()

# 2) aggregate majority per date,symbol
counts = stock.groupby(['date','symbol'])['recommendation'].value_counts().unstack(fill_value=0)

# determine majority with tie->Hold
def majority(row):
    if row.empty:
        return 'Hold'
    top_count = row.max()
    tops = row[row==top_count].index.tolist()
    if len(tops)==1:
        return tops[0]
    return 'Hold'

major = counts.apply(majority, axis=1).rename('signal').reset_index()
major['signal'] = major['signal'].where(major['signal'].isin(['Buy','Sell']), 'Hold')

# 3) merge with price data
# price CSV has columns: date,company,symbol,open,close
price.rename(columns={c:c.lower() for c in price.columns}, inplace=True)
price['date'] = pd.to_datetime(price['date']).dt.date
major['date'] = pd.to_datetime(major['date']).dt.date

merged = major.merge(price[['date','symbol','open','close']], on=['date','symbol'], how='left')

# 4) simulate trades
trades = []
for idx, r in merged.iterrows():
    sig = r['signal']
    if sig not in ['Buy','Sell']:
        continue
    if pd.isna(r['open']) or pd.isna(r['close']):
        # skip missing prices
        continue
    open_p = float(r['open'])
    close_p = float(r['close'])
    if open_p==0:
        continue
    if sig=='Buy':
        ret = (close_p - open_p)/open_p
    else:
        ret = (open_p - close_p)/open_p
    pnl = ret * CAPITAL_PER_TRADE
    trades.append({
        'date': r['date'].isoformat(),
        'symbol': r['symbol'],
        'signal': sig,
        'open': open_p,
        'close': close_p,
        'return_pct': ret,
        'pnl_usd': pnl
    })

trades_df = pd.DataFrame(trades)

out_file = os.path.join(os.getcwd(), f'analysis_results_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv')
trades_df.to_csv(out_file, index=False)

# 5) summary
print(f'Trades executed: {len(trades_df)}')
if not trades_df.empty:
    total_pnl = trades_df['pnl_usd'].sum()
    avg_pnl = trades_df['pnl_usd'].mean()
    win_rate = (trades_df['pnl_usd']>0).mean()
    print(f'Total PnL: ${total_pnl:.2f} | Avg PnL per trade: ${avg_pnl:.2f} | Win rate: {win_rate:.2%}')
    per_sym = trades_df.groupby('symbol').agg(trades=('pnl_usd','count'), pnl_sum=('pnl_usd','sum'), win_rate=('pnl_usd',lambda x: (x>0).mean())).sort_values('pnl_sum', ascending=False)
    print('\nPer-symbol summary:')
    print(per_sym)
    display(trades_df.head(30))
else:
    print('No trades executed (no Buy/Sell majority signals or missing prices).')

print('\nSaved results to', out_file)
